In [3]:
import os
import sys
import numpy as np
import gc
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset
from data.NoisyDataset import NoisyDataset, FullNoisyDataset

# Global configurations

In [4]:
#total training epoches
EPOCH = 15

SAME_PARAM = False           # to use the same parameters for all pcoders or not
FF_START = True             # to start from feedforward initialization
MAX_TIMESTEP = 5

In [1]:
# Dataset configuration
BATCH_SIZE = 10
NUM_WORKERS = 2
noise_types = ['Babble2Spkr'] #['AudScene', 'Babble8Spkr', 'SpeakerShapedNoise']
snr_levels = [-9., -6., -3.,  0.,  3.]

In [2]:
# Path names
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'

# Load network arguments

In [5]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'

In [12]:
fb_state_dict_path = f'{checkpoints_dir}{pnet_name}/{pnet_name}-50-regular.pth'
fb_state_dict = torch.load(fb_state_dict_path)

# Helper functions

In [6]:
def load_pnet(
        net, state_dict, build_graph, random_init,
        ff_multiplier, fb_multiplier, er_multiplier,
        same_param, device='cuda:0'):
    
    if same_param:
        raise Exception('Not implemented!')
    else:
        pnet = PNetClass(
            net, build_graph=build_graph, random_init=random_init,
            ff_multiplier=ff_multiplier, fb_multiplier=fb_multiplier, er_multiplier=er_multiplier
            )

    pnet.load_state_dict(state_dict)
    pnet.eval()
    pnet.to(device)
    return pnet

In [7]:
def evaluate(net, epoch, dataloader, timesteps, loss_function, writer=None, tag='Clean'):
    test_loss = np.zeros((timesteps+1,))
    correct   = np.zeros((timesteps+1,))
    for (images, labels) in dataloader:
        images = images.cuda()
        labels = labels.cuda()
        
        with torch.no_grad():
            for tt in range(timesteps+1):
                if tt == 0:
                    outputs, _ = net(images)
                else:
                    outputs, _ = net()
                
                loss = loss_function(outputs, labels)
                test_loss[tt] += loss.item()
                _, preds = outputs.max(1)
                correct[tt] += preds.eq(labels).sum()

    print()
    for tt in range(timesteps+1):
        test_loss[tt] /= len(dataloader.dataset)
        correct[tt] /= len(dataloader.dataset)
        print('Test set t = {:02d}: Average loss: {:.4f}, Accuracy: {:.4f}'.format(
            tt,
            test_loss[tt],
            correct[tt]
        ))
        if writer is not None:
            writer.add_scalar(
                f"{tag}Perf/Epoch#{epoch}",
                correct[tt], tt
                )
    print()


In [8]:
def train(net, epoch, dataloader, timesteps, loss_function, optimizer, writer=None):
    for batch_index, (images, labels) in enumerate(dataloader):
        net.reset()

        labels = labels.cuda()
        images = images.cuda()

        ttloss = np.zeros((timesteps+1))
        optimizer.zero_grad()

        for tt in range(timesteps+1):
            if tt == 0:
                outputs, _ = net(images)
                loss = loss_function(outputs, labels)
                ttloss[tt] = loss.item()
            else:
                outputs, _ = net()
                current_loss = loss_function(outputs, labels)
                ttloss[tt] = current_loss.item()
                loss += current_loss
        
        loss.backward()
        optimizer.step()
        net.update_hyperparameters()
            
        print(f"Training Epoch: {epoch} [{batch_index * 16 + len(images)}/{len(dataloader.dataset)}]\tLoss: {loss.item():0.4f}\tLR: {optimizer.param_groups[0]['lr']:0.6f}")
        for tt in range(timesteps+1):
            print(f'{ttloss[tt]:0.4f}\t', end='')
        print()
        if writer is not None:
            writer.add_scalar(
                f"TrainingLoss/CE", loss.item(),
                (epoch-1)*len(dataloader) + batch_index
                )


In [9]:
def log_hyper_parameters(net, epoch, sumwriter, same_param=True):
    if same_param:
        sumwriter.add_scalar(f"HyperparamRaw/feedforward", getattr(net,f'ff_part').item(), epoch)
        sumwriter.add_scalar(f"HyperparamRaw/feedback",    getattr(net,f'fb_part').item(), epoch)
        sumwriter.add_scalar(f"HyperparamRaw/error",       getattr(net,f'errorm').item(), epoch)
        sumwriter.add_scalar(f"HyperparamRaw/memory",      getattr(net,f'mem_part').item(), epoch)

        sumwriter.add_scalar(f"Hyperparam/feedforward", getattr(net,f'ffm').item(), epoch)
        sumwriter.add_scalar(f"Hyperparam/feedback",    getattr(net,f'fbm').item(), epoch)
        sumwriter.add_scalar(f"Hyperparam/error",       getattr(net,f'erm').item(), epoch)
        sumwriter.add_scalar(f"Hyperparam/memory",      1-getattr(net,f'ffm').item()-getattr(net,f'fbm').item(), epoch)
    else:
        for i in range(1, net.number_of_pcoders+1):
            sumwriter.add_scalar(f"Hyperparam/pcoder{i}_feedforward", getattr(net,f'ffm{i}').item(), epoch)
            if i < net.number_of_pcoders:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_feedback", getattr(net,f'fbm{i}').item(), epoch)
            else:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_feedback", 0, epoch)
            sumwriter.add_scalar(f"Hyperparam/pcoder{i}_error", getattr(net,f'erm{i}').item(), epoch)
            if i < net.number_of_pcoders:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_memory",      1-getattr(net,f'ffm{i}').item()-getattr(net,f'fbm{i}').item(), epoch)
            else:
                sumwriter.add_scalar(f"Hyperparam/pcoder{i}_memory",      1-getattr(net,f'ffm{i}').item(), epoch)


# Main hyperparameter optimization script

In [10]:
def train_and_eval(noise_type, snr_level):
    # Load clean and noisy data
    clean_ds_path = f'{engram_dir}training_dataset_random_order.hdf5'
    clean_ds = CleanSoundsDataset(clean_ds_path)
    clean_loader = torch.utils.data.DataLoader(
        clean_ds,  batch_size=BATCH_SIZE,
        shuffle=False, drop_last=False, num_workers=NUM_WORKERS
        )

    noisy_ds = NoisyDataset(bg=noise_type, snr=snr_level)
    noise_loader = torch.utils.data.DataLoader(
        noisy_ds,  batch_size=BATCH_SIZE,
        shuffle=True, drop_last=False,
        num_workers=NUM_WORKERS
        )

    # Set up logs and network for training
    net_dir = f'hyper_{noise_type}_snr{snr_level}'
    if FF_START:
        net_dir += '_FFstart'
    if SAME_PARAM:
        net_dir += '_shared'

    sumwriter = SummaryWriter(f'{tensorboard_dir}{net_dir}')
    net = BranchedNetwork() # Load original network
    net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))
    pnet_fw = load_pnet( # Load FF PNet
        net, fb_state_dict, build_graph=False, random_init=(not FF_START),
        ff_multiplier=1.0, fb_multiplier=0.0, er_multiplier=0.0,
        same_param=SAME_PARAM, device='cuda:0'
        )
    loss_function = torch.nn.CrossEntropyLoss()
    evaluate(
        pnet_fw, 0, noise_loader, 1,
        loss_function,
        writer=sumwriter, tag='FeedForward')
    del pnet_fw
    gc.collect()

    # Load PNet for hyperparameter optimization
    net = BranchedNetwork()
    net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))
    pnet = load_pnet(
        net, fb_state_dict, build_graph=True, random_init=(not FF_START),
        ff_multiplier=0.33, fb_multiplier=0.33, er_multiplier=0.0,
        same_param=SAME_PARAM, device='cuda:0'
        )

    # Set up loss function and hyperparameters
    loss_function = torch.nn.CrossEntropyLoss()
    hyperparams = [*pnet.get_hyperparameters()]
    if SAME_PARAM:
        optimizer = optim.Adam([
            {'params': hyperparams[:-1], 'lr':0.01},
            {'params': hyperparams[-1:], 'lr':0.0001}], weight_decay=0.00001)
    else:
        fffbmem_hp = []
        erm_hp = []
        for pc in range(pnet.number_of_pcoders):
            fffbmem_hp.extend(hyperparams[pc*4:pc*4+3])
            erm_hp.append(hyperparams[pc*4+3])
        optimizer = torch.optim.Adam([
            {'params': fffbmem_hp, 'lr':0.01},
            {'params': erm_hp, 'lr':0.0001}], weight_decay=0.00001)

    # Log initial hyperparameter and eval values
    log_hyper_parameters(pnet, 0, sumwriter, same_param=SAME_PARAM)
    hps = pnet.get_hyperparameters_values()
    print(hps)
    evaluate(
        pnet, 0, noise_loader,
        MAX_TIMESTEP, loss_function,
        writer=sumwriter, tag='Noisy'
        )

    # Run epochs
    for epoch in range(1, EPOCH+1):
        train(
            pnet, epoch, noise_loader,
            MAX_TIMESTEP, loss_function, optimizer,
            writer=sumwriter
            )
        log_hyper_parameters(pnet, epoch, sumwriter, same_param=SAME_PARAM)
        hps = pnet.get_hyperparameters_values()
        print(hps)

        evaluate(
            pnet, epoch, noise_loader,
            MAX_TIMESTEP, loss_function,
            writer=sumwriter, tag='Noisy'
            )
    # evaluate(
    #     pnet, epoch, clean_loader,
    #     timesteps=MAX_TIMESTEP, writer=sumwriter,
    #     tag='Clean'
    #     )
    sumwriter.close()

In [13]:
for noise_type in noise_types:
    for snr_level in snr_levels:
        print("=====================")
        print(f'{noise_type}, for SNR {snr_level}')
        print("=====================")
        train_and_eval(noise_type, snr_level)

Babble2Spkr, for SNR -9.0


/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "
/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/predify/modules/base.py:260: UserWarning: Using a target size (torch.Size([10, 164, 400])) that is different to the input size (torch.Size([10, 1, 164, 400])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  self.prediction_error  = nn.functional.mse_loss(self.prd, target)
/home/es3773/.conda/envs/hcnn/lib/python3.6/site-packages/predify/modules/base.py:2


Test set t = 00: Average loss: 0.7686, Accuracy: 0.0685
Test set t = 01: Average loss: 0.6871, Accuracy: 0.0703

[0.30000001192092896, 0.30000001192092896, 0.3999999761581421, 0.009999999776482582, 0.30000001192092896, 0.30000001192092896, 0.3999999761581421, 0.009999999776482582, 0.30000001192092896, 0.30000001192092896, 0.3999999761581421, 0.009999999776482582, 0.30000001192092896, 0.30000001192092896, 0.3999999761581421, 0.009999999776482582, 0.30000001192092896, 0.0, 0.699999988079071, 0.009999999776482582]

Test set t = 00: Average loss: 0.7686, Accuracy: 0.0685
Test set t = 01: Average loss: 0.6872, Accuracy: 0.0703
Test set t = 02: Average loss: 0.6241, Accuracy: 0.0721
Test set t = 03: Average loss: 0.5920, Accuracy: 0.0773
Test set t = 04: Average loss: 0.5866, Accuracy: 0.0844
Test set t = 05: Average loss: 0.6041, Accuracy: 0.0844

Training Epoch: 1 [10/569]	Loss: 42.3507	LR: 0.010000
8.3115	7.4629	6.8658	6.4816	6.4753	6.7535	
Training Epoch: 1 [26/569]	Loss: 38.4413	LR: 0.

Training Epoch: 2 [186/569]	Loss: 38.6514	LR: 0.010000
7.6604	6.7020	6.1146	6.0222	6.0428	6.1094	
Training Epoch: 2 [202/569]	Loss: 37.8525	LR: 0.010000
8.0200	7.0953	6.2285	5.6626	5.4492	5.3969	
Training Epoch: 2 [218/569]	Loss: 35.8558	LR: 0.010000
7.6931	6.5617	5.7497	5.3514	5.2136	5.2863	
Training Epoch: 2 [234/569]	Loss: 45.3071	LR: 0.010000
9.7139	8.3165	7.3148	6.8025	6.5436	6.6157	
Training Epoch: 2 [250/569]	Loss: 37.7404	LR: 0.010000
7.1287	6.3921	5.9733	5.8833	5.9983	6.3647	
Training Epoch: 2 [266/569]	Loss: 38.9778	LR: 0.010000
8.1234	7.1202	6.3578	5.8972	5.7100	5.7692	
Training Epoch: 2 [282/569]	Loss: 43.2525	LR: 0.010000
8.2765	7.5442	7.0083	6.7812	6.7515	6.8907	
Training Epoch: 2 [298/569]	Loss: 35.7470	LR: 0.010000
6.4148	5.6884	5.4735	5.5800	5.9124	6.6778	
Training Epoch: 2 [314/569]	Loss: 38.2355	LR: 0.010000
8.2416	7.0528	6.1667	5.6784	5.5394	5.5566	
Training Epoch: 2 [330/569]	Loss: 47.4261	LR: 0.010000
10.4968	8.8383	7.5615	7.0042	6.7613	6.7640	
Training Epoch: 2 [

Training Epoch: 3 [506/569]	Loss: 40.1153	LR: 0.010000
8.5041	7.1349	6.2472	6.0206	6.0463	6.1622	
Training Epoch: 3 [522/569]	Loss: 35.7960	LR: 0.010000
7.5709	6.3763	5.7088	5.3934	5.3031	5.4435	
Training Epoch: 3 [538/569]	Loss: 37.0211	LR: 0.010000
7.4690	6.4636	5.9104	5.6648	5.6462	5.8671	
Training Epoch: 3 [554/569]	Loss: 37.2683	LR: 0.010000
7.2708	6.4754	6.0029	5.8063	5.7235	5.9893	
Training Epoch: 3 [570/569]	Loss: 37.3490	LR: 0.010000
7.6036	6.6131	5.9597	5.7531	5.6884	5.7312	
Training Epoch: 3 [586/569]	Loss: 43.9236	LR: 0.010000
10.1041	8.2311	7.0243	6.4599	6.1257	5.9784	
Training Epoch: 3 [602/569]	Loss: 35.7644	LR: 0.010000
7.8766	6.4375	5.5094	5.1816	5.2107	5.5486	
Training Epoch: 3 [618/569]	Loss: 38.6379	LR: 0.010000
7.9913	6.8449	6.1577	5.8452	5.8334	5.9655	
Training Epoch: 3 [634/569]	Loss: 36.8855	LR: 0.010000
7.1904	6.2535	5.7877	5.7291	5.8685	6.0563	
Training Epoch: 3 [650/569]	Loss: 39.2384	LR: 0.010000
9.3288	7.4488	6.2693	5.6711	5.4098	5.1106	
Training Epoch: 3 [

...

Training Epoch: 10 [26/566]	Loss: 13.5932	LR: 0.010000
2.6775	2.4948	2.3062	2.1389	2.0143	1.9616	
Training Epoch: 10 [42/566]	Loss: 24.6552	LR: 0.010000
4.7007	4.4084	4.1299	3.9128	3.7736	3.7299	
Training Epoch: 10 [58/566]	Loss: 7.1974	LR: 0.010000
0.9524	0.9760	1.0651	1.2064	1.3903	1.6072	
Training Epoch: 10 [74/566]	Loss: 24.8155	LR: 0.010000
5.2825	4.7560	4.2043	3.7729	3.4683	3.3315	
Training Epoch: 10 [90/566]	Loss: 12.7720	LR: 0.010000
2.0899	2.0233	2.0109	2.0723	2.1981	2.3775	
Training Epoch: 10 [106/566]	Loss: 12.9177	LR: 0.010000
2.2064	2.1355	2.0815	2.0655	2.1322	2.2966	
Training Epoch: 10 [122/566]	Loss: 15.7425	LR: 0.010000
2.8364	2.6640	2.5334	2.4885	2.5406	2.6796	
Training Epoch: 10 [138/566]	Loss: 15.6954	LR: 0.010000
2.5993	2.5408	2.4928	2.5211	2.6586	2.8828	
Training Epoch: 10 [154/566]	Loss: 16.9909	LR: 0.010000
2.8905	2.7860	2.7191	2.7332	2.8424	3.0197	
Training Epoch: 10 [170/566]	Loss: 16.8562	LR: 0.010000
2.8486	2.7473	2.6958	2.7181	2.8306	3.0157	
Training Epoch: 

Training Epoch: 11 [330/566]	Loss: 15.8005	LR: 0.010000
3.1586	2.8616	2.5906	2.4233	2.3683	2.3982	
Training Epoch: 11 [346/566]	Loss: 24.3839	LR: 0.010000
5.0635	4.5756	4.1342	3.7864	3.5017	3.3225	
Training Epoch: 11 [362/566]	Loss: 21.6881	LR: 0.010000
4.5877	4.1455	3.6819	3.3097	3.0471	2.9161	
Training Epoch: 11 [378/566]	Loss: 13.8264	LR: 0.010000
2.5871	2.4562	2.3143	2.1976	2.1342	2.1371	
Training Epoch: 11 [394/566]	Loss: 17.4165	LR: 0.010000
3.2963	3.0218	2.7981	2.7028	2.7269	2.8705	
Training Epoch: 11 [410/566]	Loss: 20.7552	LR: 0.010000
3.7224	3.5380	3.4126	3.3510	3.3387	3.3926	
Training Epoch: 11 [426/566]	Loss: 15.5386	LR: 0.010000
2.5461	2.4641	2.4574	2.5455	2.6750	2.8505	
Training Epoch: 11 [442/566]	Loss: 17.0188	LR: 0.010000
3.1466	2.9639	2.8280	2.7146	2.6668	2.6989	
Training Epoch: 11 [458/566]	Loss: 22.9040	LR: 0.010000
4.3708	4.0397	3.7836	3.6071	3.5322	3.5705	
Training Epoch: 11 [474/566]	Loss: 5.7234	LR: 0.010000
0.7317	0.7576	0.8255	0.9465	1.1241	1.3380	
Training Ep

Training Epoch: 12 [634/566]	Loss: 21.9546	LR: 0.010000
3.9943	3.7632	3.5792	3.4992	3.5051	3.6137	
Training Epoch: 12 [650/566]	Loss: 18.4837	LR: 0.010000
3.3047	3.1045	2.9805	2.9802	3.0231	3.0906	
Training Epoch: 12 [666/566]	Loss: 17.0977	LR: 0.010000
2.9993	2.8314	2.7451	2.7405	2.8075	2.9740	
Training Epoch: 12 [682/566]	Loss: 19.7442	LR: 0.010000
3.9754	3.6174	3.2824	3.0442	2.9211	2.9037	
Training Epoch: 12 [698/566]	Loss: 21.9394	LR: 0.010000
3.8774	3.6791	3.5647	3.5150	3.5701	3.7332	
Training Epoch: 12 [714/566]	Loss: 22.3010	LR: 0.010000
4.6396	4.1498	3.7161	3.4004	3.2152	3.1798	
Training Epoch: 12 [730/566]	Loss: 20.6215	LR: 0.010000
3.9229	3.6437	3.4094	3.2580	3.1927	3.1947	
Training Epoch: 12 [746/566]	Loss: 25.8581	LR: 0.010000
5.0884	4.7064	4.3457	4.0661	3.8767	3.7748	
Training Epoch: 12 [762/566]	Loss: 8.4639	LR: 0.010000
1.1575	1.1822	1.2575	1.3928	1.6042	1.8696	
Training Epoch: 12 [778/566]	Loss: 22.1446	LR: 0.010000
4.3041	3.9871	3.7167	3.5007	3.3502	3.2858	
Training Ep


Test set t = 00: Average loss: 0.3352, Accuracy: 0.4523
Test set t = 01: Average loss: 0.3133, Accuracy: 0.4505
Test set t = 02: Average loss: 0.2954, Accuracy: 0.4488
Test set t = 03: Average loss: 0.2855, Accuracy: 0.4505
Test set t = 04: Average loss: 0.2839, Accuracy: 0.4452
Test set t = 05: Average loss: 0.2906, Accuracy: 0.4452

Training Epoch: 14 [10/566]	Loss: 10.4356	LR: 0.010000
1.7481	1.7144	1.6650	1.6570	1.7371	1.9140	
Training Epoch: 14 [26/566]	Loss: 15.1325	LR: 0.010000
2.4274	2.3806	2.3947	2.4757	2.6162	2.8379	
Training Epoch: 14 [42/566]	Loss: 26.8183	LR: 0.010000
5.0985	4.7733	4.4884	4.2742	4.1270	4.0570	
Training Epoch: 14 [58/566]	Loss: 25.5911	LR: 0.010000
5.0961	4.7123	4.3239	4.0114	3.7768	3.6706	
Training Epoch: 14 [74/566]	Loss: 17.6981	LR: 0.010000
3.2189	3.0100	2.8745	2.8194	2.8419	2.9334	
Training Epoch: 14 [90/566]	Loss: 13.2899	LR: 0.010000
2.1205	2.0576	2.0660	2.1688	2.3326	2.5443	
Training Epoch: 14 [106/566]	Loss: 10.2897	LR: 0.010000
1.9164	1.7743	1.65

Training Epoch: 15 [250/566]	Loss: 21.8957	LR: 0.010000
3.9623	3.7570	3.5915	3.5129	3.5123	3.5597	
Training Epoch: 15 [266/566]	Loss: 8.3357	LR: 0.010000
1.2739	1.2301	1.2134	1.3119	1.5108	1.7955	
Training Epoch: 15 [282/566]	Loss: 5.2270	LR: 0.010000
0.7782	0.7499	0.7611	0.8292	0.9659	1.1427	
Training Epoch: 15 [298/566]	Loss: 15.2902	LR: 0.010000
2.6431	2.5237	2.4423	2.4534	2.5451	2.6828	
Training Epoch: 15 [314/566]	Loss: 22.1617	LR: 0.010000
3.8154	3.6585	3.5714	3.5973	3.6994	3.8197	
Training Epoch: 15 [330/566]	Loss: 12.3860	LR: 0.010000
1.9355	1.9228	1.9706	2.0404	2.1619	2.3548	
Training Epoch: 15 [346/566]	Loss: 9.8507	LR: 0.010000
1.5427	1.5214	1.5457	1.6098	1.7219	1.9092	
Training Epoch: 15 [362/566]	Loss: 18.8717	LR: 0.010000
3.6039	3.3007	3.0747	2.9604	2.9402	2.9918	
Training Epoch: 15 [378/566]	Loss: 19.6500	LR: 0.010000
3.6669	3.4070	3.2021	3.0881	3.0875	3.1985	
Training Epoch: 15 [394/566]	Loss: 12.1170	LR: 0.010000
2.0391	1.9449	1.9139	1.9461	2.0555	2.2174	
Training Epoc